In [28]:
import pandas as pd
import spacy
import stanza
import spacy
from stanza.utils.conll import CoNLL
import os

In [29]:
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse')
nlp_spacy = spacy.load("es_core_news_md")

2025-03-03 21:10:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-03 21:10:41 INFO: Downloaded file to /home/ivan/stanza_resources/resources.json
2025-03-03 21:10:43 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-03-03 21:10:43 INFO: Using device: cpu
2025-03-03 21:10:43 INFO: Loading: tokenize
2025-03-03 21:10:44 INFO: Loading: mwt
2025-03-03 21:10:44 INFO: Loading: pos
2025-03-03 21:10:47 INFO: Loading: lemma
2025-03-03 21:10:49 INFO: Loading: depparse
2025-03-03 21:10:50 INFO: Loading: ner
2025-03-03 21:10:54 INFO: Done loading processors!


In [43]:
baselineFilePath = "/home/ivan/Desktop/Uni/TFG/RepositorioTFG/Datasets/UD_Spanish-AnCora/es_ancora-ud-test.conllu"
baseline = CoNLL.conll2doc(baselineFilePath)

if baselineFilePath[len(baselineFilePath)-6:] != ".conll":
    os.system(f"python conllu-to-conll-standard.py {baselineFilePath} {os.path.abspath("")+"/corpus.conll"}")
    os.system(f"bash callMaltParser.sh {os.path.abspath("")+"/corpus.conll"}")
else:
    os.system(f"bash callMaltParser.sh {baselineFilePath}")


sentences = [i.text for i in baseline.sentences[:100]]
malt_parser = CoNLL.conll2doc(os.path.abspath("")+"/maltParserOut.conll")
spacy_output = nlp_spacy("".join(sentences))
stanza_output = nlp_stanza("".join(sentences))

/home/ivan/Desktop/Uni/TFG/RepositorioTFG/corpus.conll


-----------------------------------------------------------------------------
                          MaltParser 1.9.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Mon Mar 03 21:35:05 CET 2025
  Transition system    : Non-Projective
  Parser configuration : Covington with allow_root=true and allow_shift=false
  Data Format          : /espmalt-1.0/conllx.xml
.          	      1	      2s	    137MB
.          	     10	      3s	    167MB
.          	    100	      4s	    179MB
..........	   1000	     15s	    150MB
.......    	   1721	     20s	    130MB
Parsing time: 00:00:19 (19505 ms)
Finished: Mon Mar 03 21:35

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/maltParserOut.conll


In [44]:
i = 1
j = 0
print(list(spacy_output.sents)[j][i].text, ":", list(spacy_output.sents)[j][i].dep_)
print(stanza_output.sentences[j].words[i].text, ":", stanza_output.sentences[j].words[i].deprel)
print(baseline.sentences[j].words[i].text, ":", baseline.sentences[j].words[i].deprel)
print(malt_parser.sentences[j].words[i].text, ":", malt_parser.sentences[j].words[i].deprel)

de : case
de : case
de : case
de : SUBJ


In [36]:
print(len(spacy_output))
print(len(list(stanza_output.iter_words())))
print(len(list(baseline.iter_words())))
print(len(list(malt_parser.iter_words())))

2984
3068
53622
53622
